In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()
    #.config("spark.some.config.option", "some-value") \

# spark is an existing SparkSession
df_inspection = spark.read.csv("./la-restaurant-market-health-data/restaurant-and-market-health-inspections.csv", 
                               header=True)
# Displays the content of the DataFrame to stdout
# df_inspection.show()
df_inspection.select('facility_name').show()
df_inspection.printSchema()

+--------------------+
|       facility_name|
+--------------------+
| HABITAT COFFEE SHOP|
|            REILLY'S|
|      STREET CHURROS|
|   TRINITI ECHO PARK|
|              POLLEN|
|      THE SPOT GRILL|
|    7 ELEVEN #37215A|
|                 CPK|
|            PHO LALA|
|          ABC DONUTS|
|            UPSTAIRS|
|       EL POLLO LOCO|
|   POONG NYUN BAKERY|
|EMC RESTAURANT GR...|
|      POPULAR DONUTS|
|   CUSCATLECA BAKERY|
|FOOD COURT TRAY W...|
|          HMS BOUNTY|
|     7 ELEVEN #27069|
|      AN'S CHAN BANG|
+--------------------+
only showing top 20 rows

root
 |-- serial_number: string (nullable = true)
 |-- activity_date: string (nullable = true)
 |-- facility_name: string (nullable = true)
 |-- score: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- service_code: string (nullable = true)
 |-- service_description: string (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- facility_address: string (nullable = true)
 |-- facility_city: st

In [2]:
df_violation = spark.read.csv("./la-restaurant-market-health-data/restaurant-and-market-health-violations.csv",
                             header=True)
df_violation.show()
df_violation.printSchema()

+-------------+--------------------+--------------------+--------------+---------------------+-----------------+------+-----+-------------------+-------------+-----------+--------------+------------+-----------+---------+-------------------+--------------------+------------------+--------------------+--------------+---------+-----+------------+-------------------+-------------+
|serial_number|       activity_date|       facility_name|violation_code|violation_description| violation_status|points|grade|   facility_address|facility_city|facility_id|facility_state|facility_zip|employee_id| owner_id|         owner_name|      pe_description|program_element_pe|        program_name|program_status|record_id|score|service_code|service_description|       row_id|
+-------------+--------------------+--------------------+--------------+---------------------+-----------------+------+-----+-------------------+-------------+-----------+--------------+------------+-----------+---------+-----------------

In [3]:
from operator import add

rdd = df_inspection.rdd.map(tuple)
count = rdd.count()
q1 = rdd.map(lambda x:x[2]).distinct().count()
q2 = rdd.map(lambda x:x[3]).map(lambda x:(9 if int(x)>99 else int(x)/10,1)).reduceByKey(add) \
    .sortByKey().map(lambda x: '[90,100]:%d' % x[1] if x[0]>=9 else '[%d,%d]:%d' % (x[0], x[0]+9, x[1])).collect()
q3 = rdd.map(lambda x:x[3]).map(lambda x:(9 if int(x)>99 else int(x)/10,1)).reduceByKey(add) \
    .sortByKey(ascending=False).map(lambda x: 'A:%f' % (x[1]*100.0/count) if x[0]>=9 else '%c:%f' % (chr(ord('A')+9-x[0]), x[1]*100.0/count)).collect()
q4 = rdd.map(lambda x:x[10]).distinct().subtract(df_violation.rdd.map(lambda x:x[10]).distinct()).count()
answer = {}
answer['count'] = count
answer['a1'] = q1
answer['a2'] = '\n'.join(q2)
answer['a3'] = '\n'.join(q3)
answer['a4'] = q4
answer

IllegalArgumentException: u'Unsupported class file major version 55'